# Marketing Campaign Results

source: https://platform.stratascratch.com/data-projects/marketing-campaign-results

In [1]:
import pandas as pd

### Readin data and initial processsing

In [ ]:
client = pd.read_csv("./datasets/client_data.csv")
deposit = pd.read_csv("./datasets/deposit_data.csv")
calendar = pd.read_csv("./datasets/calendar_data.csv")

In [8]:
print(client.info())
print(deposit.info())
print(calendar.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46347 entries, 0 to 46346
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   46347 non-null  int64 
 1   client_geographical_region  46347 non-null  object
 2   client_residence_status     46347 non-null  object
 3   client_age                  46347 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480394 entries, 0 to 480393
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   client_id        480394 non-null  int64  
 1   deposit_type     480394 non-null  object 
 2   deposit_amount   480394 non-null  float64
 3   deposit_cadence  480394 non-null  object 
 4   deposit_date     480394 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 18.3+ MB

#### Combine three data sets into one master dataset

In [ ]:
master = deposit.merge(client, on="client_id", how="left").merge(
    calendar, left_on="deposit_date", right_on="gregorian_date", how="left"
)

#### Data exploration

Can a client's desposit cadence change?
Answer: yes

In [ ]:
x = (
    deposit.sort_values(["client_id", "deposit_cadence"])
    .groupby("client_id", as_index=False)
    .deposit_cadence.unique()
    .assign(deposit_cadence=lambda x: x.deposit_cadence.apply(tuple))
)
x.deposit_cadence.value_counts()

deposit_cadence
(Monthly,)                    29271
(Biweekly,)                   11766
(Extra, Monthly)               3048
(Biweekly, Extra)              1142
(Extra,)                        845
(Biweekly, Monthly)             212
(Biweekly, Extra, Monthly)       63
Name: count, dtype: int64

An example of client switching cadence and also making extra payment

In [90]:
master.query("client_id == 945195860266").sort_values(["deposit_date", "deposit_type"])

,client_id,deposit_type,deposit_amount,deposit_cadence,deposit_date,client_geographical_region,client_residence_status,client_age,gregorian_date,month_name
60653,945195860266,Actual Deposit,348.0,Monthly,2019-06-16,West,Rent,34,2019-06-16,Month 1
60651,945195860266,Scheduled Deposit,348.0,Monthly,2019-06-16,West,Rent,34,2019-06-16,Month 1
166676,945195860266,Actual Deposit,174.0,Monthly,2019-07-20,West,Rent,34,2019-07-20,Month 2
166675,945195860266,Scheduled Deposit,174.0,Monthly,2019-07-20,West,Rent,34,2019-07-20,Month 2
60652,945195860266,Actual Deposit,174.0,Extra,2019-07-27,West,Rent,34,2019-07-27,Month 2
207569,945195860266,Actual Deposit,174.0,Biweekly,2019-08-10,West,Rent,34,2019-08-10,Month 3
207568,945195860266,Scheduled Deposit,174.0,Biweekly,2019-08-10,West,Rent,34,2019-08-10,Month 3
60655,945195860266,Actual Deposit,174.0,Biweekly,2019-08-24,West,Rent,34,2019-08-24,Month 3
60654,945195860266,Scheduled Deposit,174.0,Biweekly,2019-08-24,West,Rent,34,2019-08-24,Month 3
60646,945195860266,Actual Deposit,174.0,Biweekly,2019-09-07,West,Rent,34,2019-09-07,Month 4


Can the Scheduled deposit and actual deposit be different?
Answer: yes

In [ ]:
(
    master.query("deposit_type.isin(['Actual Deposit', 'Scheduled Deposit'])")
    .pivot_table(
        index=["client_id", "deposit_date"],
        columns="deposit_type",
        values="deposit_amount",
    )
    .dropna()
    .rename(
        {"Actual Deposit": "Actual_Deposit", "Scheduled Deposit": "Scheduled_Deposit"},
        axis=1,
    )
    .query("Actual_Deposit != Scheduled_Deposit")
)

deposit_type                  Actual_Deposit  Scheduled_Deposit
client_id       deposit_date                                   
625882515630    2019-10-20              97.0             133.00
1211140273733   2019-08-09             322.0             261.00
                2019-08-23             322.0             261.00
1274168007988   2019-10-06             223.0             246.00
2035251420486   2019-10-10             560.0             620.00
...                                      ...                ...
993234555530702 2019-10-22              25.0             190.50
997135926563943 2019-08-12             375.0             418.75
                2019-08-26             375.0             418.75
997567464814353 2019-06-26             100.0            1026.00
999904264600900 2019-08-02             418.0             835.00

[854 rows x 2 columns]

In [ ]:
catVars = master.select_dtypes("object").columns.to_list()

for v in catVars:
    print(f"{v}: {master[v].nunique()} {master[v].unique()[0:10]}")

deposit_type: 2 ['Actual Deposit' 'Scheduled Deposit']
deposit_cadence: 3 ['Monthly' 'Biweekly' 'Extra']
deposit_date: 153 ['2019-10-23' '2019-09-23' '2019-06-23' '2019-07-23' '2019-08-23'
 '2019-09-12' '2019-09-26' '2019-06-22' '2019-07-06' '2019-06-08']
client_geographical_region: 4 ['West' 'South' 'Northeast' 'Midwest']
client_residence_status: 2 ['Rent' 'Own']
gregorian_date: 153 ['2019-10-23' '2019-09-23' '2019-06-23' '2019-07-23' '2019-08-23'
 '2019-09-12' '2019-09-26' '2019-06-22' '2019-07-06' '2019-06-08']
month_name: 5 ['Month 5' 'Month 4' 'Month 1' 'Month 2' 'Month 3']


In [ ]:
master.query("client_id == 945195860266").sort_values("deposit_date")

,client_id,deposit_type,deposit_amount,deposit_cadence,deposit_date,client_geographical_region,client_residence_status,client_age,gregorian_date,month_name
60653,945195860266,Actual Deposit,348.0,Monthly,2019-06-16,West,Rent,34,2019-06-16,Month 1
60651,945195860266,Scheduled Deposit,348.0,Monthly,2019-06-16,West,Rent,34,2019-06-16,Month 1
166676,945195860266,Actual Deposit,174.0,Monthly,2019-07-20,West,Rent,34,2019-07-20,Month 2
166675,945195860266,Scheduled Deposit,174.0,Monthly,2019-07-20,West,Rent,34,2019-07-20,Month 2
60652,945195860266,Actual Deposit,174.0,Extra,2019-07-27,West,Rent,34,2019-07-27,Month 2
207569,945195860266,Actual Deposit,174.0,Biweekly,2019-08-10,West,Rent,34,2019-08-10,Month 3
207568,945195860266,Scheduled Deposit,174.0,Biweekly,2019-08-10,West,Rent,34,2019-08-10,Month 3
60654,945195860266,Scheduled Deposit,174.0,Biweekly,2019-08-24,West,Rent,34,2019-08-24,Month 3
60655,945195860266,Actual Deposit,174.0,Biweekly,2019-08-24,West,Rent,34,2019-08-24,Month 3
60646,945195860266,Actual Deposit,174.0,Biweekly,2019-09-07,West,Rent,34,2019-09-07,Month 4
